In [169]:
from typing import TypedDict, Annotated, Optional
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import add_messages, StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4
import os
import json

In [170]:
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")



In [171]:
llm = ChatGroq(model = "llama-3.3-70b-versatile", api_key = GROQ_API_KEY, temperature = 0.5)
response = llm.invoke("hey! how are you")

In [172]:
response


AIMessage(content="Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or topics you'd like to discuss. How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 40, 'total_tokens': 94, 'completion_time': 0.196363636, 'prompt_time': 0.001883423, 'queue_time': 0.053415516999999996, 'total_time': 0.198247059}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--3948a091-36b2-4f56-9216-c9f10d6d1434-0', usage_metadata={'input_tokens': 40, 'output_tokens': 54, 'total_tokens': 94})

In [173]:
response.content

"Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or topics you'd like to discuss. How about you? How's your day going so far?"

In [174]:
search_tool = TavilySearchResults(api_key = TAVILY_API_KEY, max_results = 4)

In [175]:
response = search_tool.invoke("What is the capital of France?")

In [176]:
response

[{'title': 'Paris facts: the capital of France in history',
  'url': 'https://home.adelphi.edu/~ca19535/page%204.html',
  'content': 'Home | Spain | Sydney | San Francisco | Paris | Las Vegas | Maui\nParis facts: Paris, the capital\n\r\n        of France\n\nParis is the capital of France,\r\n      the largest country of Europe\r\n      with 550 000 km2 (65 millions inhabitants).\n\nParis has 2.234 million inhabitants\r\n      end 2011. She is the core of Ile de France region (12 million\r\n      people). [...] Before Paris, the capital of France\r\n        was Lyon\r\n        (under the Romans). Paris first became the capital of France in\r\n        508 under King Clovis. After centuries with no unique capital of\r\n        France, Paris retrieved its status of capital of France under King\r\n          Philippe Auguste, who reigned between 1180 and 1223. You\r\n        can see remains of the Philippe August Paris walls in the\r\n        passageway between the Louvre\n\r\n          park

In [177]:
tools = [search_tool]        ## list of tools to be used in the graph

In [178]:
## initialize the memory saver

memory = MemorySaver()

In [179]:
llm_with_tools = llm.bind_tools(tools = tools)

##### Building Agentic Workflow

In [180]:
## Define the state for agent graph

class State(TypedDict):
    messages : Annotated[list, add_messages]                ## State:- Flowing the input to the entire workflow.

In [181]:
## Function 

async def model(state: State):                     ## pass the input in the form of state.

    messages = state["messages"]
    result = await llm_with_tools.ainvoke(messages)
    return {"messages": [result]}

In [182]:
from langgraph.prebuilt import ToolNode

search_tool = ToolNode([search_tool])

In [183]:
async def route_to_tools(state: State):
    
    messages = state["messages"]
    last_message = messages[-1]

    if(hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0):
        return "Search Tool"
    else: 
        return END


In [184]:
graph_workflow = StateGraph(State)

graph_workflow.add_node("Model", model)
graph_workflow.add_node("Search Tool", search_tool)

In [185]:
from langgraph.graph import START

In [186]:
graph_workflow.add_edge(START, "Model")
graph_workflow.add_conditional_edges("Model", route_to_tools)
graph_workflow.add_edge("Search Tool", "Model")


In [187]:
app = graph_workflow.compile(checkpointer = memory)

In [188]:
mermaid_code = app.get_graph(xray=True).draw_mermaid()
print(mermaid_code)


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	Model(Model)
	Search_Tool(Search Tool)
	__end__(<p>__end__</p>)
	__start__ --> Model;
	Model --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [189]:
config = {
    "configurable" : {
        "thread_id" : 2
    }
}

response = await app.ainvoke({
    "messages" : [HumanMessage(content = "what is the current situation in jammu?")]
}, config = config)

In [190]:
response

{'messages': [HumanMessage(content='what is the current situation in jammu?', additional_kwargs={}, response_metadata={}, id='1d752e33-ee41-4de6-86ff-8026f701f6ac'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7tnk', 'function': {'arguments': '{"query": "current situation in Jammu"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 287, 'total_tokens': 309, 'completion_time': 0.08, 'prompt_time': 0.023656683, 'queue_time': 0.054907175, 'total_time': 0.103656683}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6a4ad2bf-b7e4-401f-a7fa-afb19766e88f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current situation in Jammu'}, 'id': 'call_7tnk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 287, 'output_tokens': 22, 'total_tokens': 309}),
  ToolMessa

In [191]:
config = {
    "configurable" : {
        "thread_id" : 2
    }
}


async for event in app.astream_events({
    "messages": [HumanMessage(content="When is the next spacex launch?")],
}, config=config, version="v2"):
    print(event)


{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='When is the next spacex launch?', additional_kwargs={}, response_metadata={})]}}, 'name': 'LangGraph', 'tags': [], 'run_id': '30e17c4c-ece3-4661-99b5-d7fc197afde2', 'metadata': {'thread_id': 2}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='what is the current situation in jammu?', additional_kwargs={}, response_metadata={}, id='1d752e33-ee41-4de6-86ff-8026f701f6ac'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7tnk', 'function': {'arguments': '{"query": "current situation in Jammu"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 287, 'total_tokens': 309, 'completion_time': 0.08, 'prompt_time': 0.023656683, 'queue_time': 0.054907175, 'total_time': 0.103656683}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e

In [194]:
config = {
    "configurable" : {
        "thread_id" : 9
    }
}


async for event in app.astream_events({
    "messages": [HumanMessage(content="WHAT WAS MY NAME?")],
}, config=config, version="v2"):
    if event["event"] == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush = True)


Your name is BRIGU.